In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size
img_size = (469, 369)

# Load CSV files
df_train = pd.read_csv(
    "/content/Continous_Check/cont_data_train.csv"
)
df_test = pd.read_csv(
    "/content/Continous_Check/cont_data_test.csv"
)
df_val = pd.read_csv(
    "/content/Continous_Check/cont_data_val.csv"
)

# Data generator for scaling images
scale = ImageDataGenerator(rescale=1.0 / 255)

# Create train, test, and validation generators
train_generator = scale.flow_from_dataframe(
    df_train,
    x_col="Path",
    y_col=["Label","ChirpMass"],
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
    shuffle=True
)

test_generator = scale.flow_from_dataframe(
    df_test,
    x_col="Path",
    y_col=["Label","ChirpMass"],
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
    shuffle=True
)

val_generator = scale.flow_from_dataframe(
    df_val,
    x_col="Path",
    y_col=["Label","ChirpMass"],
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
    shuffle=True
)

KeyboardInterrupt: 

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def create_cnn_model(input_shape):
    inputs = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)

    # Flatten the output of the convolutional layers
    x = Flatten()(x)

    # Fully connected layers
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output layers for different parameters
    t_merger_output = Dense(1, name='t_merger', activation='linear')(x)
    chirp_mass_output = Dense(1, name='chirp_mass', activation='linear')(x)
    SNR = Dense(1,name='SNR',activation='linear')(x)

    # Define the model with the inputs and outputs
    model = Model(inputs=inputs, outputs=[t_merger_output, chirp_mass_output, SNR])

    return model


In [7]:
import tensorflow as tf
from tensorflow.keras import backend as K

def chirp_mass_loss(y_true, y_pred):

    # Calculate the mean squared error
    mse = K.mean(K.square(y_pred - y_true), axis=-1)

    # Scale the MSE by (chirp mass)^(5/3)
    chirp_mass_factor = K.pow(y_true, 5/3)

    return mse * chirp_mass_factor

In [8]:

input_shape = (469, 369, 3)
model = create_cnn_model(input_shape)

# Compile the model with appropriate loss functions and optimizer
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss={'t_merger': 'mse',
                    'sky_location': 'mse',
                    'distance': 'mse',
                    'chirp_mass': chirp_mass_loss},
                    
              metrics={'t_merger': 'mae',
                       'sky_location': 'mae',
                       'distance': 'mae',
                       'chirp_mass': 'mae'})

# Print the model summary
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 469, 369, 3)]        0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 469, 369, 32)         896       ['input_3[0][0]']             
                                                                                                  
 max_pooling2d_6 (MaxPoolin  (None, 234, 184, 32)         0         ['conv2d_6[0][0]']            
 g2D)                                                                                             
                                                                                                  
 batch_normalization_6 (Bat  (None, 234, 184, 32)         128       ['max_pooling2d_6[0][0]'